In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install allennlp
from allennlp.models.bimpm import BiMpm

     |████████████████████████████████| 7.6MB 2.8MB/s 
     |████████████████████████████████| 256kB 44.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 163kB 46.6MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 133kB 48.8MB/s 
     |████████████████████████████████| 194kB 53.3MB/s 
     |████████████████████████████████| 245kB 55.0MB/s 
     |████████████████████████████████| 1.0MB 49.2MB/s 
     |████████████████████████████████| 645kB 50.1MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.14.0-cp36-cp36m-linux_x86_64.whl size=3320380 sha256=46a86c579e36da6c352acc6aeff9b59341b508db032d027b0077163a3ec43524
  Stored in directory: /root/.cache/pip/wheels/5b/b7/83/985f0f758fbb34f14989a0fab86d18890d1cc5ae12f26967bc
  Created wheel for ftfy: filename=ftfy-5.6-cp36-none-any.whl size=44553 sha256=8b40e6b2e96ab0f892af15c8375727289d903774b4c064cacd749538d1b7a62b
  Stored in directory: /roo

In [0]:
import logging

# Importing Datasets


In [0]:
import torch
from torchtext import data
from torchtext.vocab import Vectors

class BQ():
    def __init__(self, args):
        self.RAW = data.RawField()
        self.RAW.is_target = False
        tokenize = lambda x : list(x)
        self.TEXT = data.Field(batch_first=True, tokenize = tokenize)
        self.LABEL = data.Field(sequential=False, unk_token=None)

        self.train, self.dev, self.test = data.TabularDataset.splits(
            path='dataset/sent_pair/bq',
            train='train.tsv',
            validation='dev.tsv',
            test='test.tsv',
            format='tsv',
            fields=[('label', self.LABEL),
                    ('q1', self.TEXT),
                    ('q2', self.TEXT),
                    ('id', self.RAW)])

        self.TEXT.build_vocab(self.train, self.dev, self.test, vectors=Vectors("BQ300", args.data))
        self.LABEL.build_vocab(self.train)

        sort_key = lambda x: data.interleave_keys(len(x.q1), len(x.q2))

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.train_iter, self.dev_iter, self.test_iter = \
            data.BucketIterator.splits((self.train, self.dev, self.test),
                                       batch_sizes=[args.batch_size] * 3,
                                       device=device,
                                       sort_key=sort_key)
        self.max_word_len = max([len(w) for w in self.TEXT.vocab.itos])
        # for <pad>
        self.char_vocab = {'': 0}
        # for <unk> and <pad>
        self.characterized_words = [[0] * self.max_word_len, [0] * self.max_word_len]

        if args.use_char_emb:
            self.build_char_vocab()